In [0]:
#Mounting Google Drive to Colab Pro
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144568 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.21-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
#Mounting Google Drive to Colab Pro
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
#Testing access to Google drive
!ls /content/drive/StatNLP/FinalProject/

 ADVERSARIA-ATTACK-GENERATOR-Dheeraj.ipynb	     'Data Sets'
 AnchorTags_and_Adversarial-Attacks-Generator.ipynb  'Model Files'
 Blackbox.ipynb					      README.md


## **Master Variables**

In [0]:
#Data Storage Parameters
dataset_dir = "/content/drive/StatNLP/FinalProject/Data Sets/asap-aes"
adversarial_dir = "/content/drive/StatNLP/FinalProject/Data Sets/adversarial_asap"
model_save_dir = "/content/drive/StatNLP/FinalProject/Model Files"
selected_essay_id = 2
training_set_file = dataset_dir+"/training_set_rel3.xls" 

###Test sets:
test_set_file = dataset_dir+"/valid_set.xls"
test_set_scores_file = dataset_dir+"/valid_sample_submission_5_column.csv"

#Data Embedding Parameters
# Take First X words from each essay, abandon rest
max_len = 1118 #longest essay 

# Word Dimensionality - consider the top 15,000 words in the dataset
max_words = 20000

In [0]:

def make_prediction(modelname,sampess):
    sample_prediction = modelname.predict(test_set_essays_emb[sampess:sampess+1])
    return sample_prediction
  
def calculate_score(prediction):
    score = {}
    score[1]=prediction[0,0]
    score[2]=prediction[0,1]
    score[3]=prediction[0,2]    
    score[4]=prediction[0,3]  
    score[5]=prediction[0,4]    
    score[6]=prediction[0,5]    
    calculate_score = max(score, key=score.get)
    return(calculate_score)

# **Loading Packages and Dependencies**

In [0]:
#data loading
import os

# python modules
from argparse import Namespace
from collections import Counter
import json
import re
import string
import statistics

####data manipulation####
import numpy as np
from numpy.random import shuffle 
import pandas as pd

####word2vec encoding####
import gensim

####data visualization####
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
%matplotlib inline
plt.style.use('ggplot')

####CNN tools####

#keras
import keras
from keras import layers
from keras import models
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras.models import load_model
from keras import regularizers
from keras import metrics
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


## ** Load and Clean Dataset**

In [0]:

#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
#verify data paths
print(training_set_file)
print(test_set_file)

#load excel into dataframe
raw_training_set = pd.read_excel(training_set_file, sheet_name='training_set')
test_set = pd.read_excel(test_set_file, sheet_name='Sheet1')
test_set_scores = pd.read_csv(test_set_scores_file)

print("\nEntire Corpus for ASAP:")
print("Training Set:",raw_training_set.shape)
#print("Validation:",valid_set.shape)
print("Test Set:",test_set.shape,"\n")

#filter data by essay set
essay_fltr =  raw_training_set['essay_set']== selected_essay_id
training_set = raw_training_set[essay_fltr]

essay_fltr =  test_set['essay_set']== selected_essay_id
test_set = test_set[essay_fltr]

essay_fltr =  test_set_scores['essay_set']== selected_essay_id
test_set_scores = test_set_scores[essay_fltr]

#remove empty n/a cells
training_set = training_set.drop(['rater3_domain1','rater1_trait1','rater1_trait2','rater1_trait3','rater1_trait4','rater1_trait5','rater1_trait6','rater2_trait1','rater2_trait2','rater2_trait3','rater2_trait4','rater2_trait5','rater2_trait6','rater3_trait1','rater3_trait2','rater3_trait3','rater3_trait4','rater3_trait5','rater3_trait6'], axis=1)    
test_set = test_set.drop(['domain2_predictionid'], axis=1)    

training_set_top = training_set.head()
#print(training_set_top)
test_set_top = test_set.head()
#print(test_set_top)

#3 sets, training, validation and testing

print("Selected Essay Set #%s Corpus:" % selected_essay_id)
print("Training Set:",training_set.shape)
print("Test Set:",test_set.shape)
print("Total Data Set:", training_set.shape[0]+test_set.shape[0])

/content/drive/StatNLP/FinalProject/Data Sets/asap-aes/training_set_rel3.xls
/content/drive/StatNLP/FinalProject/Data Sets/asap-aes/valid_set.xls

Entire Corpus for ASAP:
Training Set: (1800, 28)
Test Set: (600, 5) 

Selected Essay Set #2 Corpus:
Training Set: (1800, 9)
Test Set: (600, 4)
Total Data Set: 2400


Splitting into Essays and Labels

In [0]:
#extract essays and convert to NumPy for Keras
training_set_essays = training_set['essay']
training_set_essays = training_set_essays.values
test_set_essays = test_set['essay']
test_set_essays = test_set_essays.values

#extract scores and convert to NumPy for Keras
training_set_dom1scores = training_set['domain1_score']
training_set_dom1scores = training_set_dom1scores.values

#extract domain#1 predicted scores
#data cleaning due to strange score input shape
test_set_dom1scores = []
for i in (range(test_set_scores.shape[0])):
    if (i % 2) == 0: #print every other cell, since second cell is domain#2
        asdf = test_set_scores['predicted_score'].values[i]
        i_score_no = float(asdf)
        #print(asdf)
        #test_set_dom1scores = test_set_dom1scores.append({'predicted_score': asdf}, ignore_index=True)
        test_set_dom1scores.append(i_score_no)
#convert to NumPy Array
test_set_dom1scores = np.asarray(test_set_dom1scores)

## **Encoding Essays**

Tokenization and word indexing

In [0]:
# Vectorize the Essays

#TEMPORARILY COMBINE TRAIN AND TEST TO SIMPLIFY EMBEDDING PROCESS
#single embedding process, max token index
lengthmark = len(training_set_essays)
combined_essays = np.append(training_set_essays,test_set_essays)

# Tokenize the data 
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(combined_essays)
sequences = tokenizer.texts_to_sequences(combined_essays)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

# Pad sequences that are shorter than others
combined_data_pen = pad_sequences(sequences, maxlen=max_len)

#SPLIT TRAINING AND TEST SETS BACK
train_data_pen = combined_data_pen[:lengthmark]
test_data_pen = combined_data_pen[lengthmark:]

# Load the label
print('Shape of Testing data tensor:', test_data_pen.shape)

Found 17023 unique tokens.
Shape of Testing data tensor: (600, 1118)


One-hot encoding of essay scores

In [0]:
train_labels_pen = np.zeros((0, 6))

#Scores to Dummy Variable Conversion
#Training (and Validation) Set
for item in training_set_dom1scores:
      if item==1:
          train_labels_pen = np.append(train_labels_pen, [[1,0,0,0,0,0]],axis = 0)
      elif item==2:
          train_labels_pen = np.append(train_labels_pen, [[0,1,0,0,0,0]],axis = 0)        
      elif item==3:
          train_labels_pen = np.append(train_labels_pen, [[0,0,1,0,0,0]],axis = 0)        
      elif item==4:
          train_labels_pen = np.append(train_labels_pen, [[0,0,0,1,0,0]],axis = 0)        
      elif item==5:
          train_labels_pen = np.append(train_labels_pen, [[0,0,0,0,1,0]],axis = 0)        
      else:
          train_labels_pen = np.append(train_labels_pen, [[0,0,0,0,0,1]],axis = 0)   

test_labels_pen = np.zeros((0, 6))

#Scores to Dummy Variable Conversion
#Testing Set
for item in test_set_dom1scores:
      if item==1:
          test_labels_pen = np.append(test_labels_pen, [[1,0,0,0,0,0]],axis = 0)
      elif item==2:
          test_labels_pen = np.append(test_labels_pen, [[0,1,0,0,0,0]],axis = 0)        
      elif item==3:
          test_labels_pen = np.append(test_labels_pen, [[0,0,1,0,0,0]],axis = 0)        
      elif item==4:
          test_labels_pen = np.append(test_labels_pen, [[0,0,0,1,0,0]],axis = 0)        
      elif item==5:
          test_labels_pen = np.append(test_labels_pen, [[0,0,0,0,1,0]],axis = 0)        
      else:
          test_labels_pen = np.append(test_labels_pen, [[0,0,0,0,0,1]],axis = 0)   
          
print("Test Labels Shape:" ,test_labels_pen.shape)

Test Labels Shape: (600, 6)


In [0]:
#TEST SET IS LEFT ALONE
test_split = 0.80
val_set_essays = training_set_essays 
val_set_dom1scores = training_set_dom1scores

#split coded scores
set_split_test = int((len(train_data_pen))*test_split)
training_set_essays_emb, val_set_essays_emb = train_data_pen[:set_split_test], train_data_pen[set_split_test:]
training_set_dom1scores_emb, val_set_dom1scores_emb = train_labels_pen[:set_split_test], train_labels_pen[set_split_test:]
#split the unencoded scores
training_set_dom1scores, val_set_dom1scores = training_set_dom1scores[:set_split_test], training_set_dom1scores[set_split_test:]

test_set_essays_emb = test_data_pen
test_set_dom1scores_emb = test_labels_pen

print("\nTest Set Essays and matching Scores:")
print("Shape: ",test_set_essays_emb.shape, test_set_dom1scores_emb.shape)


Test Set Essays and matching Scores:
Shape:  (600, 1118) (600, 6)


## **Loadind Blackbox model**

In [0]:
test_model_black_box = load_model(model_save_dir+'/D1_76_BLACKBOX_CNN.h5')

## **Anchor Identification Model**

In [0]:

!pip install anchor_exp
!pip install -q spacy && python -m spacy download en_core_web_lg && python -m spacy link en_core_web_lg enlg

from anchor import anchor_text

import spacy
spacy_nlp = spacy.load('enlg')

     |████████████████████████████████| 430kB 4.5MB/s 
     |████████████████████████████████| 276kB 10.7MB/s 
     |████████████████████████████████| 2.0MB 9.4MB/s 
  Created wheel for anchor-exp: filename=anchor_exp-0.0.1.0-cp36-none-any.whl size=434742 sha256=69740d97daae7669675a1fd6b0ec84d0e5af81cd858256ac4289f99d19305e0d
  Stored in directory: /root/.cache/pip/wheels/ff/b7/cd/7bfb36f4a01ff6b1509cd3432f8b208f455d232cee1079e309
  Created wheel for lime: filename=lime-0.2.0.0-cp36-none-any.whl size=284181 sha256=3ff30060d0d3fa42f0ad4412d566446da5a8009a6e737532bbbc55d39e6934d8
  Stored in directory: /root/.cache/pip/wheels/22/f2/ec/e5ebd07348b2b1ac722e91c2f549fcc220f7d5f25497a61232
Successfully built anchor-exp lime
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0


     |████████████████████████████████| 827.9MB 2.0MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=fd712a060ae34b892b11556def94c97e9977d5b554fa8544c3a8cef6b33e849c
  Stored in directory: /tmp/pip-ephem-wheel-cache-j5iavqye/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_lg -->
/usr/local/lib/python3.6/dist-packages/spacy/data/enlg
You can now load the model via spacy.load('enlg')


In [0]:
sample_ids = [0]

for idx in sample_ids:
    print('Index: %d, Feature: %s' % (idx, test_set_essays[idx]))
    print('True Score: %s' % (test_set_dom1scores[idx]))
    estimatedscore=[calculate_score(make_prediction(test_model_black_box, idx))]
    def estimator(estimatedscore):
      estimator = np.asarray(estimatedscore)
      return estimator

    #classifier_fn([text])[0]

    explainer = anchor_text.AnchorText(spacy_nlp, [1,2,3,4,5,6], use_unk_distribution=True)
    exp = explainer.explain_instance(test_set_essays[idx], estimator, threshold=0.95, use_proba=True, batch_size=30)

    max_pred = 2
    print('Key Signal from Anchors: %s' % (' AND '.join(exp.names())))
    print(exp.features())
    print('Precision: %.2f' % exp.precision())
    print()

    #exp.show_in_notebook()

Index: 0, Feature: Do u believe there are books, music, magizines, and movies in are libaries? These meterials could consist of nude pictures and so on. Most parents do not want to see there kids getting a hold of this type of material. You maybe think 'what could  we do about this things, or least come to a compermise to make almost everyone happy. I have so reasons why we should put this type of material away from kids sight and reach.     First of all, the books and magizines that have any nude in them, should have their own section and, be labled aduts only. I know this might be alot of work but it is needed, so kids aren't aloud to go in ths section. This will help ruduce what little kids see. There are also books and magizines that are offensive to children of a different race. These books should be removed from the shelves also because they hert peoples feelings, and this might make customers leave your libary. I am not saying ouhae to take ever single book of your selves, but a

In [0]:

#using a real essay

sample_ids = [0]

for idx in sample_ids:
    print('Index: %d, Feature: %s' % (idx, training_set_essays[idx]))
    print('True Score: %s' % (training_set_dom1scores[idx]))
    estimatedscore=[calculate_score(make_prediction(test_model_black_box, idx))]
    def estimator(estimatedscore):
      estimator = np.asarray(estimatedscore)
      return estimator

    #classifier_fn([text])[0]

    explainer = anchor_text.AnchorText(spacy_nlp, [1,2,3,4,5,6], use_unk_distribution=True)
    exp = explainer.explain_instance(training_set_essays[idx], estimator, threshold=0.8, use_proba=True, batch_size=30)

    max_pred = 2
    print('Key Signal from Anchors: %s' % (' AND '.join(exp.names())))
    print('Precision: %.2f' % exp.precision())
    print()

    #exp.show_in_notebook()

In [0]:
!nvidia-smi

Wed Apr 22 07:18:26 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    33W / 250W |   1579MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
adversarial_dir = "/content/drive/StatNLP/FinalProject/Data Sets/adversarial_asap"
test_set_file = adversarial_dir+"/valid_set_plus_ADVERSARIAL_ESSAYS.xls"

#verify data paths
print(test_set_file)

/content/drive/StatNLP/FinalProject/Data Sets/adversarial_asap/valid_set_plus_ADVERSARIAL_ESSAYS.xls


In [0]:
import random
# Attack 1: Shuffling Words

#load excel into dataframe
test_set_shuffle = pd.read_excel(test_set_file, sheet_name='valid_set')

#remove empty n/a cells
test_set_shuffle = test_set_shuffle.drop(['domain2_predictionid'], axis=1)    


for i in test_set_shuffle.index: 
  words= test_set_shuffle.at[i, 'essay'].split()
  random.shuffle(words)
  new_sentence = ' '.join(words)
  test_set_shuffle.at[i,'essay'] = new_sentence  
  
test_set_shuffle.to_excel(adversarial_dir+"/valid_set_plus_ADVERSARIAL_ESSAYS-SHUFFLE.xls")

In [0]:
!ls /content/drive/StatNLP/FinalProject/Data Sets

ls: cannot access '/content/drive/StatNLP/FinalProject/Data': No such file or directory
ls: cannot access 'Sets': No such file or directory


In [0]:
!ls /content/drive/StatNLP/FinalProject/Data\ Sets/adversarial_asap

a
valid_sample_submission_5_column_plus_ADVERSARIAL_ESSAYS.csv
valid_set_plus_ADVERSARIAL_ESSAYS.xls


In [0]:
# Attack 2a: Appending - "Library"

#load excel into dataframe
test_set_append = pd.read_excel(test_set_file, sheet_name='valid_set')

#remove empty n/a cells
test_set_append = test_set_append.drop(['domain2_predictionid'], axis=1)    


for i in test_set_append.index: 
  words= test_set_append.at[i, 'essay'].split()
  words.append("library")
  new_sentence = ' '.join(words)
  test_set_append.at[i,'essay'] = new_sentence  
  
test_set_append.to_excel(adversarial_dir+"/valid_set_plus_ADVERSARIAL_ESSAYS-APPEND_LIBRARY.xls")

In [0]:
# Attack 3a: Progressive Overload - "Library"

#load excel into dataframe
test_set_progressive = pd.read_excel(test_set_file, sheet_name='valid_set')

#remove empty n/a cells
test_set_progressive = test_set_progressive.drop(['domain2_predictionid'], axis=1)    


for i in test_set_progressive.index: 
  words= test_set_progressive.at[i, 'essay'].split()
  if i < 591:
    continue
  if i < 641:
    for x in range(0,i-590):
      words[x] = "library"
  new_sentence = ' '.join(words)
  test_set_progressive.at[i,'essay'] = new_sentence  
  
test_set_progressive.to_excel(adversarial_dir+"/valid_set_plus_ADVERSARIAL_ESSAYS-PROGRESSIVE_LIBRARY.xls")

In [0]:
# Attack 4a: Single Substitution - "Library"

#load excel into dataframe
test_set_single = pd.read_excel(test_set_file, sheet_name='valid_set')

#remove empty n/a cells
test_set_single = test_set_single.drop(['domain2_predictionid'], axis=1)    


for i in test_set_single.index: 
  words= test_set_single.at[i, 'essay'].split()
  if i < 591:
    continue
  if i < 641:
    words[i-591] = "library"
  new_sentence = ' '.join(words)
  test_set_single.at[i,'essay'] = new_sentence  
  
test_set_single.to_excel(adversarial_dir+"/valid_set_plus_ADVERSARIAL_ESSAYS-SINGLE_LIBRARY.xls")

In [0]:
# Attack 5a: Insertion of anchor in random locations

#load excel into dataframe
test_set_insertion = pd.read_excel(test_set_file, sheet_name='valid_set')

#remove empty n/a cells
test_set_insertion = test_set_insertion.drop(['domain2_predictionid'], axis=1)    


for i in test_set_insertion.index: 
  words= test_set_insertion.at[i, 'essay'].split()
  x = random.randint(0,len(words))
  words.insert(x, 'library')
  new_sentence = ' '.join(words)
  test_set_insertion.at[i,'essay'] = new_sentence  
  
test_set_insertion.to_excel(adversarial_dir+"/valid_set_plus_ADVERSARIAL_ESSAYS-INSERTION_LIBRARY.xls")

In [0]:
# Attack 2b: Appending - "Censorship"

#load excel into dataframe
test_set_append = pd.read_excel(test_set_file, sheet_name='valid_set')

#remove empty n/a cells
test_set_append = test_set_append.drop(['domain2_predictionid'], axis=1)    


for i in test_set_append.index: 
  words= test_set_append.at[i, 'essay'].split()
  words.append("censorship")
  new_sentence = ' '.join(words)
  test_set_append.at[i,'essay'] = new_sentence  
  
test_set_append.to_excel(adversarial_dir+"/valid_set_plus_ADVERSARIAL_ESSAYS-APPEND_CENSORSHIP.xls")

In [0]:
# Attack 3b: Progressive Overload - "Censorship"

#load excel into dataframe
test_set_progressive = pd.read_excel(test_set_file, sheet_name='valid_set')

#remove empty n/a cells
test_set_progressive = test_set_progressive.drop(['domain2_predictionid'], axis=1)    


for i in test_set_progressive.index: 
  words= test_set_progressive.at[i, 'essay'].split()
  if i < 591:
    continue
  if i < 641:
    for x in range(0,i-590):
      words[x] = "censorship"
  new_sentence = ' '.join(words)
  test_set_progressive.at[i,'essay'] = new_sentence  
  
test_set_progressive.to_excel(adversarial_dir+"/valid_set_plus_ADVERSARIAL_ESSAYS-PROGRESSIVE_CENSORSHIP.xls")

In [0]:
# Attack 4b: Single Substitution - "Censorship"

#load excel into dataframe
test_set_single = pd.read_excel(test_set_file, sheet_name='valid_set')

#remove empty n/a cells
test_set_single = test_set_single.drop(['domain2_predictionid'], axis=1)    


for i in test_set_single.index: 
  words= test_set_single.at[i, 'essay'].split()
  if i < 591:
    continue
  if i < 641:
    words[i-591] = "censorship"
  new_sentence = ' '.join(words)
  test_set_single.at[i,'essay'] = new_sentence  
  
test_set_single.to_excel(adversarial_dir+"/valid_set_plus_ADVERSARIAL_ESSAYS-SINGLE_CENSORSHIP.xls")

In [0]:
# Attack 5b: Insertion of "censorship" in random locations

#load excel into dataframe
test_set_insertion = pd.read_excel(test_set_file, sheet_name='valid_set')

#remove empty n/a cells
test_set_insertion = test_set_insertion.drop(['domain2_predictionid'], axis=1)    


for i in test_set_insertion.index: 
  words= test_set_insertion.at[i, 'essay'].split()
  x = random.randint(0,len(words))
  words.insert(x, 'censorship')
  new_sentence = ' '.join(words)
  test_set_insertion.at[i,'essay'] = new_sentence  
  
test_set_insertion.to_excel(adversarial_dir+"/valid_set_plus_ADVERSARIAL_ESSAYS-INSERTION_CENSORSHIP.xls")

In [0]:
# Attack 2c: Appending - "The"

#load excel into dataframe
test_set_append = pd.read_excel(test_set_file, sheet_name='valid_set')

#remove empty n/a cells
test_set_append = test_set_append.drop(['domain2_predictionid'], axis=1)    


for i in test_set_append.index: 
  words= test_set_append.at[i, 'essay'].split()
  words.append("the")
  new_sentence = ' '.join(words)
  test_set_append.at[i,'essay'] = new_sentence  
  
test_set_append.to_excel(adversarial_dir+"/valid_set_plus_ADVERSARIAL_ESSAYS-APPEND_THE.xls")

In [0]:
# Attack 3c: Progressive Overload - "The"

#load excel into dataframe
test_set_progressive = pd.read_excel(test_set_file, sheet_name='valid_set')

#remove empty n/a cells
test_set_progressive = test_set_progressive.drop(['domain2_predictionid'], axis=1)    


for i in test_set_progressive.index: 
  words= test_set_progressive.at[i, 'essay'].split()
  if i < 591:
    continue
  if i < 641:
    for x in range(0,i-590):
      words[x] = "the"
  new_sentence = ' '.join(words)
  test_set_progressive.at[i,'essay'] = new_sentence  
  
test_set_progressive.to_excel(adversarial_dir+"/valid_set_plus_ADVERSARIAL_ESSAYS-PROGRESSIVE_THE.xls")

In [0]:
# Attack 4c: Single Substitution - "The"

#load excel into dataframe
test_set_single = pd.read_excel(test_set_file, sheet_name='valid_set')

#remove empty n/a cells
test_set_single = test_set_single.drop(['domain2_predictionid'], axis=1)    


for i in test_set_single.index: 
  words= test_set_single.at[i, 'essay'].split()
  if i < 591:
    continue
  if i < 641:
    words[i-591] = "censorship"
  new_sentence = ' '.join(words)
  test_set_single.at[i,'essay'] = new_sentence  
  
test_set_single.to_excel(adversarial_dir+"/valid_set_plus_ADVERSARIAL_ESSAYS-SINGLE_THE.xls")

In [0]:
# Attack 5c: Insertion of "the" in random locations

#load excel into dataframe
test_set_insertion = pd.read_excel(test_set_file, sheet_name='valid_set')

#remove empty n/a cells
test_set_insertion = test_set_insertion.drop(['domain2_predictionid'], axis=1)    


for i in test_set_insertion.index: 
  words= test_set_insertion.at[i, 'essay'].split()
  x = random.randint(0,len(words))
  words.insert(x, 'the')
  new_sentence = ' '.join(words)
  test_set_insertion.at[i,'essay'] = new_sentence  
  
test_set_insertion.to_excel(adversarial_dir+"/valid_set_plus_ADVERSARIAL_ESSAYS-INSERTION_THE.xls")